In [2]:
#!pip install pandas matplotlib seaborn scikit-learn

In [3]:
import pandas as pd
print("Notebook is working! 🎉")

Notebook is working! 🎉


## 1. Project Overview
- Predict customer churn for a fictional telco/banking product
- Binary classification problem (churned vs. retained)

## 2. Load & Preview Data

## 3. Data Cleaning & Preprocessing

## 4. Exploratory Data Analysis (EDA)

## 5. Feature Engineering

## 6. Model Training

## 7. Model Evaluation

## 8. Insights & Next Steps